In [1]:
import pandas as pd
import numpy as np
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
import yaml

pd.set_option("display.max_columns", 100)
%matplotlib inline

/Users/timong/anaconda/envs/py36/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
def pg_load_table(file_path, table_name, dbname, user):
    '''
    This function upload csv to a target table
    '''
    try:
        conn = psycopg2.connect(dbname=dbname, user=user)
        print("Connecting to Database")
        cur = conn.cursor()
        f = open(file_path, "r")
        # Truncate the table first
        cur.execute("Truncate {} Cascade;".format(table_name))
        print("Truncated {}".format(table_name))
        # Load table from the file with header
        cur.copy_expert("copy {} from STDIN CSV HEADER QUOTE '\"'".format(table_name), f)
        cur.execute("commit;")
        print("Loaded data into {}".format(table_name))
        conn.close()
        print("DB connection closed.")

    except Exception as e:
        print("Error: {}".format(str(e)))
        sys.exit(1)

In [3]:
with open("./postgres.yaml", "r") as stream:
    try:
        data_loaded = yaml.load(stream)
    except yaml.YAMLERROR as exc:
        print(exc)

In [4]:
dbname = data_loaded['dbname']
user = data_loaded['user']
pg_load_table('./user_transactions.csv', 'transactions', dbname, user)
pg_load_table('./user_label_branch_edit.csv', 'user_label_branch', dbname, user)
# decided to load user_base_part1 only as user_base_part2 is just a duplicate of the first (ref to below cell)
pg_load_table('./user_base_part1.csv', 'user_base', dbname, user)

Connecting to Database
Truncated transactions
Loaded data into transactions
DB connection closed.
Connecting to Database
Truncated user_label_branch
Loaded data into user_label_branch
DB connection closed.
Connecting to Database
Truncated user_base
Loaded data into user_base
DB connection closed.


outstanding: total outstanding amount of credit card usage
credit_limit: credit limit amount that can be used
total_cash_usage: last month total cash usage of customer
total_retail_usage: last month total retail usage of customer
bill: last month customer bill amount
remaining_bill: remaining bill that has not been paid in the last month (assuming that this refers to the remaining amount of > month -2 bills that was not paid in month -1)

number_of_cards: # cards owned by customer
branch_code: branch code of bank
default_flag: 1: default, 0: non_default

total_usage = assume total_cash_usage + total_retail_usage
payment_ratio: payment per bill ratio in the last month (payment made to billing amount in the last month)
payment_ratio_3month: payment per bill ratio in the last 3 month
payment_ratio_6month: payment per bill ratio in the last 6 month
overlimit_percentage: overlimit percentage

In [5]:
user1 = pd.read_csv('./user_base_part1.csv')
user2 = pd.read_csv('./user_base_part2.csv')
transactions = pd.read_csv('./user_transactions.csv')
user_label_branch = pd.read_csv('./user_label_branch_edit.csv')

In [6]:
user1.equals(user2)
# user_base_part1 dataframe is exactly the same as user_base_part2

True

## SQL 1

In [7]:
query_combine = """
select 
    label.user_id, label.number_of_cards, label.branch_code, label.default_flag,
    transactions.outstanding, transactions.credit_limit, transactions.bill, transactions.total_cash_usage, transactions.total_retail_usage, transactions.remaining_bill,
    base.payment_ratio, base.overlimit_percentage, base.payment_ratio_3month, base.payment_ratio_6month, base.deliquency_score, base.years_since_card_issuing, base.total_usage,
    base.remaining_bill_per_number_of_cards, base.remaining_bill_per_limit, base.total_usage_per_limit, base.total_3mo_usage_per_limit, base.total_6mo_usage_per_limit,
    base.utilization_3month, base.utilization_6month
from 
    user_label_branch as label
join
    transactions
on 
    label.user_id = transactions.user_id
join
    user_base as base
on 
    label.user_id = base.user_id
"""
combine = pd.read_sql(query_combine, con = psycopg2.connect(database = dbname, user = user))

In [8]:
combine.head()

,user_id,number_of_cards,branch_code,default_flag,outstanding,credit_limit,bill,total_cash_usage,total_retail_usage,remaining_bill,payment_ratio,overlimit_percentage,payment_ratio_3month,payment_ratio_6month,deliquency_score,years_since_card_issuing,total_usage,remaining_bill_per_number_of_cards,remaining_bill_per_limit,total_usage_per_limit,total_3mo_usage_per_limit,total_6mo_usage_per_limit,utilization_3month,utilization_6month
0,1,2,I,0,36158.0,7000000.0,23437.0,0.0,94.0,26323.0,102.19,0.0,74.78,100.00,0.0,15.416667,94.0,13161.5,0.003760,0.000013,0.011719,0.017810,0.013228,0.021949
1,2,2,A,0,268691.0,10000000.0,254564.0,0.0,1012.0,0.0,0.00,0.0,0.00,0.00,NaN,0.750000,1012.0,0.0,0.000000,0.000101,0.000000,0.000000,0.004232,0.000300
2,3,3,A,0,6769149.0,28000000.0,4159779.0,0.0,0.0,0.0,100.00,0.0,100.00,100.91,NaN,10.750000,0.0,0.0,0.000000,0.000000,0.040518,0.047703,0.249389,0.267853
3,4,4,G,0,3496732.0,21000000.0,111231.0,0.0,2536660.0,581334.0,100.00,0.0,25.01,22.64,0.0,19.750000,2536660.0,145333.5,0.027683,0.120793,0.055971,0.016851,0.101912,0.346635
4,5,2,A,0,9402085.0,10000000.0,6099283.0,0.0,2666558.0,5951865.0,95.99,0.0,97.49,99.84,NaN,1.666667,2666558.0,2975932.5,0.595186,0.266656,0.323027,0.131162,0.707865,0.336571


## SQL 2

In [9]:
query_default_stats = """
with main as (
select 
    label.user_id, label.number_of_cards, label.branch_code, label.default_flag,
    transactions.outstanding, transactions.credit_limit, transactions.bill, transactions.total_cash_usage, transactions.total_retail_usage, transactions.remaining_bill,
    base.payment_ratio, base.overlimit_percentage, base.payment_ratio_3month, base.payment_ratio_6month, base.deliquency_score, base.years_since_card_issuing, base.total_usage,
    base.remaining_bill_per_number_of_cards, base.remaining_bill_per_limit, base.total_usage_per_limit, base.total_3mo_usage_per_limit, base.total_6mo_usage_per_limit,
    base.utilization_3month, base.utilization_6month
from 
    user_label_branch as label
join
    transactions
on 
    label.user_id = transactions.user_id
join
    user_base as base
on 
    label.user_id = base.user_id
)
select
    default_flag,
    branch_code,
    count(user_id) as user_count,
    avg(deliquency_score) as avg_deliquency_score,
    percentile_disc(0.5) within group (order by deliquency_score) as median_deliquency_score,
    avg(number_of_cards) as avg_no_cards,
    percentile_disc(0.5) within group (order by number_of_cards) as median_no_cards,
    avg(years_since_card_issuing) as avg_years_since_card_issuing,
    percentile_disc(0.5) within group (order by years_since_card_issuing) as median_years_since_card_issuing,
    avg(outstanding) as avg_outstanding,
    percentile_disc(0.5) within group (order by outstanding) as median_outstanding,
    avg(credit_limit) as avg_credit_limit,
    percentile_disc(0.5) within group (order by credit_limit) as median_credit_limit,
    avg(bill) as avg_bill,
    percentile_disc(0.5) within group (order by bill) as median_bill,
    avg(remaining_bill) as avg_remaining_bill,
    percentile_disc(0.5) within group (order by remaining_bill) as median_remaining_bill,
    avg(remaining_bill_per_number_of_cards) as avg_remaining_bill_per_card,
    percentile_disc(0.5) within group (order by remaining_bill_per_number_of_cards) as median_remaining_bill_per_number_of_cards,
    avg(remaining_bill_per_limit) as avg_remaining_bill_per_limit,
    percentile_disc(0.5) within group (order by remaining_bill_per_limit) as median_remaining_bill_per_limit,
    avg(total_cash_usage) as avg_total_cash_usage,
    percentile_disc(0.5) within group (order by total_cash_usage) as median_total_cash_usage,
    avg(total_retail_usage) as avg_total_retail_usage,
    percentile_disc(0.5) within group (order by total_retail_usage) as median_total_retail_usage,
    avg(total_usage) as avg_total_usage,
    percentile_disc(0.5) within group (order by total_usage) as median_total_usage,
    avg(total_usage_per_limit) as avg_total_usage_per_limit,
    percentile_disc(0.5) within group (order by total_usage_per_limit) as median_total_usage_per_limit,
    avg(total_3mo_usage_per_limit) as avg_total_3mo_usage_per_limit,
    percentile_disc(0.5) within group (order by total_3mo_usage_per_limit) as median_total_3mo_usage_per_limit,
    avg(total_6mo_usage_per_limit) as avg_total_6mo_usage_per_limit,
    percentile_disc(0.5) within group (order by total_6mo_usage_per_limit) as median_total_6mo_usage_per_limit,
    avg(payment_ratio) as avg_payment_ratio,
    percentile_disc(0.5) within group (order by payment_ratio) as median_payment_ratio,
    avg(payment_ratio_3month) as avg_payment_ratio_3mo,
    percentile_disc(0.5) within group (order by payment_ratio_3month) as median_payment_ratio_3mo,
    avg(payment_ratio_6month) as avg_payment_ratio_6mo,
    percentile_disc(0.5) within group (order by payment_ratio_6month) as median_payment_ratio_6mo,
    avg(overlimit_percentage) as avg_overlimit_percentage,
    percentile_disc(0.5) within group (order by overlimit_percentage) as median_overlimit_percentage,
    avg(utilization_3month) as avg_3mo_utilization,
    percentile_disc(0.5) within group (order by utilization_3month) as median_3mo_utilization,
    avg(utilization_6month) as avg_6mo_utilization,
    percentile_disc(0.5) within group (order by utilization_6month) as median_6mo_utilization
from main
group by 1, 2
order by 2, 1
"""
default_stats = pd.read_sql(query_default_stats, con = psycopg2.connect(database = dbname, user = user))
# can use min, max, percentile_disc(0.25) and percentile_disc(0.75) functions as well

In [10]:
default_stats

,default_flag,branch_code,user_count,avg_deliquency_score,median_deliquency_score,avg_no_cards,median_no_cards,avg_years_since_card_issuing,median_years_since_card_issuing,avg_outstanding,median_outstanding,avg_credit_limit,median_credit_limit,avg_bill,median_bill,avg_remaining_bill,median_remaining_bill,avg_remaining_bill_per_card,median_remaining_bill_per_number_of_cards,avg_remaining_bill_per_limit,median_remaining_bill_per_limit,avg_total_cash_usage,median_total_cash_usage,avg_total_retail_usage,median_total_retail_usage,avg_total_usage,median_total_usage,avg_total_usage_per_limit,median_total_usage_per_limit,avg_total_3mo_usage_per_limit,median_total_3mo_usage_per_limit,avg_total_6mo_usage_per_limit,median_total_6mo_usage_per_limit,avg_payment_ratio,median_payment_ratio,avg_payment_ratio_3mo,median_payment_ratio_3mo,avg_payment_ratio_6mo,median_payment_ratio_6mo,avg_overlimit_percentage,median_overlimit_percentage,avg_3mo_utilization,median_3mo_utilization,avg_6mo_utilization,median_6mo_utilization
0,0,A,7926,0.001020,0.0,2.556523,2,6.801886,5.750000,1.287613e+07,5502098.0,2.475858e+07,11000000.0,8.833918e+06,3564668.0,8.689746e+06,2943538.0,3.162557e+06,1265266.000,0.435955,0.268000,72852.422786,0.0,2.550122e+06,386423.0,2.623016e+06,437000.0,0.110354,0.034400,0.159605,0.103000,0.185342,0.110000,68.618142,36.01,57.181604,54.30,84.539334,66.70,2.645548,0.00,0.543604,0.526000,0.506983,0.453000
1,1,A,767,0.035202,0.0,2.646675,2,6.834343,5.330000,2.221352e+07,6581960.0,2.861799e+07,13000000.0,1.715552e+07,5527047.0,1.924337e+07,5708855.0,6.428818e+06,2675706.000,0.693199,0.850000,112908.829205,0.0,7.822129e+05,0.0,8.951217e+05,0.0,0.034665,0.000000,0.125564,0.054800,0.223138,0.096200,18.657862,0.00,15.137184,20.20,37.914003,29.10,8.125671,0.00,0.720949,0.868000,0.682328,0.750000
2,0,B,1216,0.000822,0.0,2.438322,2,6.441246,5.500000,8.524250e+06,3392930.0,1.477508e+07,6000000.0,6.100867e+06,2480236.0,5.376491e+06,2061332.0,2.064334e+06,943965.000,0.459918,0.329000,138006.226974,0.0,1.878351e+06,150000.0,2.016488e+06,192000.0,0.129818,0.024300,0.184489,0.122821,0.219093,0.121000,56.624359,25.10,72.476727,52.20,78.262714,66.70,3.223873,0.00,0.562567,0.575000,0.530451,0.511000
3,1,B,132,0.000000,0.0,2.424242,2,6.464015,5.000000,9.706657e+06,3586261.0,1.521970e+07,6000000.0,7.930416e+06,3338474.0,7.824128e+06,3232414.0,2.864681e+06,1477983.000,0.604379,0.747779,129166.666667,0.0,4.758968e+05,0.0,6.050635e+05,0.0,0.030750,0.000000,0.127516,0.057919,0.170329,0.086800,16.732273,0.00,35.322955,25.00,49.792879,33.30,7.446591,0.00,0.623807,0.748000,0.573085,0.537000
4,0,C,305,0.006557,0.0,2.413115,2,6.445442,5.670000,7.265405e+06,3632373.0,1.496066e+07,7000000.0,5.674350e+06,3186854.0,5.316427e+06,2589638.0,2.162015e+06,1178209.000,0.468860,0.308000,34081.967213,0.0,1.169826e+06,272379.0,1.203908e+06,307820.0,0.113672,0.032000,0.181225,0.130000,0.217267,0.151000,64.686754,31.00,64.156951,53.40,68.493443,66.70,2.999803,0.00,0.567405,0.599381,0.530460,0.515000
5,1,C,27,0.000000,0.0,2.296296,2,5.812222,5.250000,8.422023e+06,4027195.0,1.385185e+07,7000000.0,7.070592e+06,3487608.0,6.811747e+06,3682477.0,2.682734e+06,1841239.000,0.689242,1.040000,35185.185185,0.0,8.090296e+05,0.0,8.442147e+05,0.0,0.056509,0.000000,0.143865,0.081300,0.229708,0.146000,17.541481,0.00,56.719630,16.91,86.610370,46.20,8.276296,4.36,0.725468,0.927000,0.651004,0.819000
6,0,D,193,0.005181,0.0,2.466321,2,6.830174,5.670000,8.158524e+06,4645467.0,1.459585e+07,7000000.0,5.954875e+06,2408082.0,6.327167e+06,2739377.0,2.335120e+06,1280569.000,0.466917,0.406000,37046.632124,0.0,9.026852e+05,96000.0,9.397319e+05,100000.0,0.098800,0.009030,0.172793,0.098945,0.237442,0.138000,68.188290,28.80,51.799171,49.70,54.455544,60.80,2.894611,0.00,0.577920,0.640000,0.572731,0.588000
7,1,D,12,3.000000,3.0,2.500000,2,6.553306,4.666667,5.998939e+06,3929654.0,1.458333e+07,5000000.0,5.686584e+06,4179179.0,5.003234e+06,3661225.0,1.699731e+06,1830613.000,0.586711,0.732000,NaN,NaN,2.500417e+05,0.0

## SQL 3

In [11]:
# check branch_code variant
query_check_bc = """
select distinct
    branch_code
from user_label_branch
"""
check_branch_code = pd.read_sql(query_check_bc, con = psycopg2.connect(database = dbname, user = user))

In [12]:
check_branch_code

,branch_code
0,None
1,b
2,i
3,j
4,H
5,e
6,J
7,C
8,D
9,I


There are null values in branch code and there is a mix of upper and lower case. 

In [13]:
print("number of null values in branch_code column: " + str(len(user_label_branch[user_label_branch.branch_code.isnull()])))
print("% of rows where branch_code is null: " + str(round(len(user_label_branch[user_label_branch.branch_code.isnull()])/user_label_branch.shape[0]*100, 1)) + '%')

number of null values in branch_code column: 195
% of rows where branch_code is null: 1.2%


In [14]:
# change upper case branch code to lower case
user_label_branch = (
    user_label_branch
    .pipe(lambda x: x.assign(branch_code=x.branch_code.str.lower()))
)

In [15]:
user_label_branch.branch_code.describe()

count     15450
unique       11
top           a
freq       8705
Name: branch_code, dtype: object

In [16]:
# replace null values with 'a' as it occurs the most often
user_label_branch.loc[user_label_branch[user_label_branch.branch_code.isnull()].index.values, 'branch_code'] = 'a'

In [17]:
# reload user_label_branch into psql
user_label_branch.to_csv('./user_label_branch_edit.csv', index=False)
pg_load_table('./user_label_branch_edit.csv', 'user_label_branch', dbname, user)

Connecting to Database
Truncated user_label_branch
Loaded data into user_label_branch
DB connection closed.


In [18]:
# use 'usage' as a proxy to number of transactions completed. assuming higher the usage, the more the number of transactions completed
# since 6 months usage data is available, I shall provide the top 5 users with the most number of 'transactions' from each 'branch code'
# to get total_usage_6months, I will multiply 'total_6mo_usage_per_limit' with 'credit_limit'
# check 'total_6mo_usage_per_limit' and 'credit_limit' for anomaly
combine[['total_6mo_usage_per_limit', 'credit_limit']].describe()

,total_6mo_usage_per_limit,credit_limit
count,15645.000000,1.564500e+04
mean,0.202454,2.082010e+07
std,0.274421,2.955419e+07
min,-0.569000,3.000000e+06
25%,0.032600,5.000000e+06
50%,0.117000,9.000000e+06
75%,0.285000,2.200000e+07
max,8.110000,1.000000e+09


In [19]:
# there are negative values for total_6mo_usage_per_limit
print("number of entries where total_6mo_usage_per_limit is negative: " + str(len(combine.query("total_6mo_usage_per_limit < 0"))))

number of entries where total_6mo_usage_per_limit is negative: 17


In [20]:
combine.query("total_6mo_usage_per_limit < 0")[['total_usage_per_limit', 'total_6mo_usage_per_limit']]

,total_usage_per_limit,total_6mo_usage_per_limit
40,0.000000,-0.081559
208,0.010286,-0.001212
1423,0.000000,-0.000152
3371,0.090400,-0.012500
4866,0.005380,-0.009210
5226,0.007330,-0.000325
7482,0.063800,-0.037700
8943,0.000000,-0.025600
9639,0.000000,-0.009000
10122,0.000000,-0.137000


In [21]:
# assuming there cannot be negative values for 'total_6mo_usage_per_limit
# from the above dataframe, we can see that 'total_usage_per_limit' and 'total_6mo_usage_per_limit are similar, other than the negative sign
# to clean the data we shall take the absolute of existing value
user1 = (
    user1
    .pipe(lambda x: x.assign(total_6mo_usage_per_limit=abs(x.total_6mo_usage_per_limit)))
)

In [22]:
# reload user_base into psql
user1.to_csv('./user_base_part1_edit.csv', index=False)
pg_load_table('./user_base_part1_edit.csv', 'user_base', dbname, user)

Connecting to Database
Truncated user_base
Loaded data into user_base
DB connection closed.


In [23]:
query_top5 = """
with main as (
select
    base.user_id,
    base.total_6mo_usage_per_limit * t.credit_limit as total_usage_6month,
    label.branch_code
from user_base as base
join transactions as t
on base.user_id = t.user_id
join user_label_branch as label
on base.user_id = label.user_id
)
select
    branch_code,
    rank,
    user_id,
    total_usage_6month
from
    (select
        user_id,
        total_usage_6month,
        branch_code,
        rank() over (partition by branch_code order by total_usage_6month desc) as rank
    from main
    ) as tmp
where rank <= 5
order by 1, 2
"""
top5 = pd.read_sql(query_top5, con = psycopg2.connect(database = dbname, user = user))

In [24]:
top5

,branch_code,rank,user_id,total_usage_6month
0,a,1,9597,8.570000e+08
1,a,2,13236,4.980000e+08
2,a,3,10550,4.543000e+08
3,a,4,11041,4.541600e+08
4,a,5,4056,4.288000e+08
5,b,1,14061,1.434000e+08
6,b,2,9096,1.245000e+08
7,b,3,1442,7.323522e+07
8,b,4,15208,5.187000e+07
9,b,5,2813,5.083000e+07


## SQL 4

In [25]:
query_decile = """
with main as (
select
    t.user_id,
    label.default_flag,
    t.outstanding,
    ntile(10) over (order by t.outstanding asc) AS decile
FROM transactions as t
join user_label_branch as label
on t.user_id = label.user_id
),
intermediate as (
select 
    decile,
    count(user_id) as user_count,
    sum(default_flag) as default_count,
    cast(sum(default_flag) as float)/cast(count(user_id) as float) as default_rate,
    min(outstanding) as min_outstanding,
    avg(outstanding) as avg_outstanding,
    max(outstanding) as max_outstanding
from main
group by 1
)
select
    decile,
    user_count,
    default_count,
    default_rate,
    sum(default_rate) over (order by decile) as cumulative_default_rate,
    min_outstanding,
    avg_outstanding,
    max_outstanding
from intermediate
order by 1
"""
decile = pd.read_sql(query_decile, con = psycopg2.connect(database = dbname, user = user))

In [26]:
decile

,decile,user_count,default_count,default_rate,cumulative_default_rate,min_outstanding,avg_outstanding,max_outstanding
0,1,1565,344,0.219808,0.219808,0.0,1.321829e+05,334128.0
1,2,1565,53,0.033866,0.253674,334750.0,7.923163e+05,1334327.0
2,3,1565,48,0.030671,0.284345,1334725.0,1.985958e+06,2587708.0
3,4,1565,97,0.061981,0.346326,2587715.0,3.058284e+06,3463385.0
4,5,1565,101,0.064537,0.410863,3463542.0,4.042975e+06,4721563.0
5,6,1564,138,0.088235,0.499098,4722525.0,5.346846e+06,6028571.0
6,7,1564,113,0.072251,0.571349,6029687.0,7.188113e+06,8494395.0
7,8,1564,132,0.084399,0.655748,8496119.0,1.099222e+07,14730155.0
8,9,1564,153,0.097826,0.753574,14730155.0,2.000410e+07,27377448.0
9,10,1564,237,0.151535,0.905108,27377448.0,6.254995e+07,798058574.0
